<font size = "12">Compsci 361 Assignment 1</font>

<font size = "5"> Name: Joanne Chen </font>

<font size = "5">UPI: jehc820 </font>

In [505]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 
#print(y)

# metadata 
print(mushroom.metadata) 
  
# variable information 
print(mushroom.variables)

{'uci_id': 73, 'name': 'Mushroom', 'repository_url': 'https://archive.ics.uci.edu/dataset/73/mushroom', 'data_url': 'https://archive.ics.uci.edu/static/public/73/data.csv', 'abstract': 'From Audobon Society Field Guide; mushrooms described in terms of physical characteristics; classification: poisonous or edible', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 8124, 'num_features': 22, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['poisonous'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1981, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5959T', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': "This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525).  Each species is identified as definitely edible, definitely po

In [506]:
import pandas as pd
import numpy as np
import math
from scipy import stats
import matplotlib.pyplot as plt
#from matplotlib import pyplot
from sklearn.model_selection import train_test_split


<font size = "5">TASK A & B </font>

We firstly replace all missing data with the majority class.

In [507]:
# #calculate amount of missing value in the dataset
missing_value = X.isna().sum()
missing_value_count = missing_value.sum()
print("mushroom missing values: ", missing_value_count)
#there are 2480 missing value


mushroom missing values:  2480


In [508]:
#replace the missing value with mode
for fea in X:
    if X[fea].isna().sum() != 0:
        mode = X[fea].value_counts().idxmax()
        X[fea] = X[fea].fillna(mode)

#test
print("mushroom missing values: ", X.isna().sum().sum())
#print(X["stalk-root"])
print(X["stalk-root"].value_counts())

mushroom missing values:  0
stalk-root
b    6256
e    1120
c     556
r     192
Name: count, dtype: int64


/var/folders/kh/qlpm2bl96z120l3yf51tl2_m0000gn/T/ipykernel_2822/805927011.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[fea] = X[fea].fillna(mode)


<font size = '3'>To begin, we import the mushroom dataset. Then we read all the dataset, investigare missing data, and find out the majority class to replace all the missing values. We only found missing values in "stalk-root" column with 2480 missing data using .isna().sum(). We use for loop to find the mode in the column. In our case, 'b' is the mode. Then we replace all '?' with 'b'. Finally, we test the missing value again using .value_counts(). </font>

In [509]:
#Calculates the entropy of the given data set
#lables is a list of lables
def cal_entropy(labels):
    #total amount of lables
    total_label = len(labels)

    #if there is only one lable then entropy is 1
    if total_label <= 1:
        return 0
    
    #counting the amount of each lable
    if len(np.unique(labels)) == 1:
        return 0
    counts = labels.value_counts()
    count_of_e = counts['e']
    count_of_p = counts['p']

    #calculate probs of e and p
    probs_e = count_of_e / total_label
    probs_p = count_of_p / total_label

    #return entropy
    return -1 * (probs_e * math.log(probs_e,2) + probs_p * math.log(probs_p,2))

#test with y
#print(cal_entropy(y))

#Calculates the information gain after splitting the data on the chosen attribute. 
#attr (attributes) is a part of X and labels is a part of y
def ig(attr, labels, col_index):
    #calculate entropy before split
    entropy_b4_split = cal_entropy(labels)

    #calculate the sum of entropy after split
    total_attr = len(labels)

    #find the unique value and its amount in the each column
    uniq_fea = np.unique(attr.iloc[:, col_index])
    attr_counts = attr.iloc[:, col_index].value_counts()
    
    #calculate sum of entropy after split
    entropy_after_split = 0
    for element in uniq_fea:
        label_after_split = labels[attr.iloc[:, col_index] == element] #array of labels after split
        entropy_after_split += attr_counts[element] / total_attr * cal_entropy(label_after_split)

    return entropy_b4_split - entropy_after_split

#test with X and y
print(ig(ms_dataset_X_train,ms_dataset_y_train,3))

0.1955123731414904


<font size = '3'> In this part, we use NumPY and Pandas to define entropy and information gain method. These two method will be used in the DecisionTreeClassifier.</font>

In [510]:
#we calculate the info gain of all attributes (all columns) recursively. 
#We will pick the attribute with highest score for the first split. 

def build_decision_tree(X, y, current_depth, stopping_depth):
    max_ig = -1
    best_fea_idx = 0 #record the index of attribute with the highest score of ig

    #get best feature with highest ig
    num_row, num_col = X.shape
    for col_idx in range(num_col):
        fea_ig = ig(X, y, col_idx)
        if fea_ig > max_ig:
            max_ig = fea_ig
            best_fea_idx = col_idx

    #find the best attribute column
    best_fea_df = X.iloc[:,best_fea_idx]
    best_feature = X.columns[best_fea_idx]

    #seperate all row number(index) by unique attributes
    uniq_fea_list = np.unique(best_fea_df)

    #if only one column left or
    #if we go thru all features or 
    #if reach to the target depth (depth control) then stop recursion and return the predicted result
    if (len(uniq_fea_list) <= 1) or (num_col == 0) or (current_depth > stopping_depth):
        labels_array, label_count = np.unique(y, return_counts=True) #labels_array is an array of 'e' and 'p'
        return labels_array[np.argmax(label_count)]  #return 'e' or 'p' by investigating which one appear more frequently

    #start to build the decision tree; make a dictionary to store the root
    result_tree_dict = {best_feature:{}}

    for fea in uniq_fea_list:

        #find all row number of a type of unique feature
        satisfied_index = X.index[best_fea_df == fea].tolist() 

        #drop the used column? #X_sub = X.drop(columns=[best_feature]).loc[satisfied_index] #eg all 'a' stumps in the best col
        X_sub = X.loc[satisfied_index]
        y_sub = y.loc[satisfied_index]
        #list and tuple method #result_tree_dict[best_feature].append((fea, build_decision_tree(X_sub, y_sub, current_depth+1, stopping_depth)))
        result_tree_dict[best_feature][fea] = build_decision_tree(X_sub, y_sub, current_depth+1, stopping_depth)
        #use dict to store n-ary tree stumps

    return result_tree_dict


#TEST
#print(split_data(X,y))
print(build_decision_tree(X,y,0,1))


{'odor': {'a': {'cap-shape': {'b': 'e', 'f': 'e', 'x': 'e'}}, 'c': 'p', 'f': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}, 'l': {'cap-shape': {'b': 'e', 'f': 'e', 'x': 'e'}}, 'm': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}, 'n': {'spore-print-color': {'b': 'e', 'h': 'e', 'k': 'e', 'n': 'e', 'o': 'e', 'r': 'p', 'w': 'e', 'y': 'e'}}, 'p': {'cap-shape': {'f': 'p', 'x': 'p'}}, 's': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}, 'y': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}}}


We implement a decision tree using information gain to pick the best feature. Then we seperate the dataframe into different subset based on the different attribute, like 'a', 'f'. Then we recursively calculate the best feature with the highest information gain.

We also have depth control in this build_decision_tree method. Stopping_depth is a parameter of this method, so we can input whatever depth or level we want to stop the recursion (calculation).

<font size = "5">TASK C </font>

In [511]:
#split dataset into train and test data
ms_dataset_X_train, ms_dataset_X_test, ms_dataset_y_train, ms_dataset_y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
#print(ms_dataset_X_train)
#print(ms_dataset_X_train.to_numpy())

In [512]:
feature_name_list = list(mushroom.variables.name)[1:]
print(feature_name_list)

['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


In [516]:
trained_tree = build_decision_tree(ms_dataset_X_train,ms_dataset_y_train,0,1)
print(trained_tree)

{'odor': {'a': {'cap-shape': {'b': 'e', 'f': 'e', 'x': 'e'}}, 'c': 'p', 'f': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}, 'l': {'cap-shape': {'b': 'e', 'f': 'e', 'x': 'e'}}, 'm': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}, 'n': {'spore-print-color': {'b': 'e', 'h': 'e', 'k': 'e', 'n': 'e', 'o': 'e', 'r': 'p', 'w': 'e', 'y': 'e'}}, 'p': {'cap-shape': {'f': 'p', 'x': 'p'}}, 's': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}, 'y': {'cap-shape': {'f': 'p', 'k': 'p', 'x': 'p'}}}}


<font size = '3'> In this part, we split data into training data and test data. 80% of the total data is training data and 20% is the test dataset. I choose this method to split the data because the data is big enough and 80%/20% method that only run the test once is the fastest and easiest way to test the data.</font>

In [519]:
def get_instance_predict(tree, test_instance): #test_instance is a pd Series object
    if isinstance(tree, dict):  #if subtree is dict, we continue, otherwise it is label
        feature_name = list(tree.keys())[0] #feature, 'odor'
        subtree = tree[feature_name] #attribute, a/f/l...
        instance = test_instance[feature_name] #find the attribute of test_instance using column name
        if instance in subtree:
            return get_instance_predict(subtree[instance], test_instance)
        else:
            return 'UNKNOWN' #return UNKNOW if there is no matched instance in the trained tree
    return tree

def get_df_predict_result(trained_tree, X_test):
    pred_result = []
    #get all feature in every row in the test data
    for index, instance in X_test.iterrows(): #iterate over each rows of the test dataframe and get the instance
        pred_result.append(get_instance_predict(tree, instance))
    return pred_result

print(get_df_predict_result(trained_tree, ms_dataset_X_test))

['e', 'p', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'p', 'e', 'e', 'p', 'p', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e',

We implement a test procedure using the training data we split before.

<font size = "5">TASK D </font>

Task D: We pick accuracy as the evaluation method for my build_decision_tree algorithm. Accuracy = (TP + TN) / (P + N)

We choose this matrix because database is big enough and this evaluation method is easy to implement.

In [523]:
y_true= ms_dataset_y_test["poisonous"].tolist()
print(y_true)

['e', 'p', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'p', 'e', 'e', 'p', 'p', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e',

In [524]:
y_test = get_df_predict_result(trained_tree, ms_dataset_X_test)
print(y_test)

['e', 'p', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'p', 'e', 'e', 'p', 'p', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e',

In [528]:
#Calculates the accuracy of the model predictions
#y_true is the list of true labels; y_test is the list of predictions
def calculate_accuracy(y_true, y_test):
    correct_counts = np.sum(np.array(y_test) == np.array(y_true))
    total = len(y_true)
    return correct_counts / total * 100

acc = calculate_accuracy(y_true,y_test)
print(acc)

99.32307692307693
